In [1]:
# The code was removed by Watson Studio for sharing.

,ICAO24,Callsign,Origin_Country,Time_Position,Last_Contact,Long,Lat,Baro_Altitude,On_Ground,Velocity,True_Track,Vertical_Rate,Sensors,Geo_Altitude,Squawk,SPI,Position_Source
0,471f8d,WZZ2581,Hungary,1658404684,1658404684,0.1860,51.6859,7620.0,False,190.83,287.90,0.0,No Data,8001.0,3114,False,0
1,407b3c,GTTEA,United Kingdom,1658404510,1658404519,-0.4454,51.6245,403.86,False,76.02,275.83,-0.98,No Data,525.78,7000,False,0
2,405455,SHT17A,United Kingdom,1658404683,1658404683,-0.2642,51.4781,571.5,False,93.12,269.68,-5.2,No Data,723.9,6231,False,0
3,40556f,GCEZI,United Kingdom,1658404683,1658404684,0.2091,51.2496,373.38,False,49.92,178.23,-0.98,No Data,487.68,3767,False,0
4,40688f,BAW248,United Kingdom,1658404683,1658404684,-0.5437,51.3322,3032.76,False,135.82,359.35,-5.53,No Data,3223.26,1426,False,0


In [2]:
#from bokeh.plotting import figure, show
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.tile_providers import get_provider,STAMEN_TERRAIN
from bokeh.models import HoverTool,LabelSet,ColumnDataSource
import numpy as np


#FUNCTION TO CONVERT GCS WGS84 TO WEB MERCATOR
#POINT
def wgs84_web_mercator_point(lon,lat):
    k = 6378137
    x= lon * (k * np.pi/180.0)
    y= np.log(np.tan((90 + lat) * np.pi/360.0)) * k
    return x,y

#DATA FRAME
def wgs84_to_web_mercator(df, lon="Long", lat="Lat"):
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

#COORDINATE CONVERSION
xy_min=wgs84_web_mercator_point(lon_min,lat_min)
xy_max=wgs84_web_mercator_point(lon_max,lat_max)
wgs84_to_web_mercator(flight_df)
flight_df['rot_angle']=flight_df['True_Track']*-1 #Rotation angle
icon_url='http://cdn.onlinewebfonts.com/svg/img_9926.png' #Icon url
flight_df['url']=icon_url


#FIGURE SETTING
x_range,y_range=([xy_min[0],xy_max[0]], [xy_min[1],xy_max[1]])
p=figure(x_range=x_range,y_range=y_range,x_axis_type='mercator',y_axis_type='mercator',sizing_mode='scale_width',plot_height=300)

#PLOT BASEMAP AND AIRPLANE POINTS
flight_source=ColumnDataSource(flight_df)
tile_prov=get_provider(STAMEN_TERRAIN)
p.add_tile(tile_prov,level='image')
p.image_url(url='url', x='x', y='y',source=flight_source,anchor='center',angle_units='deg',angle='rot_angle',h_units='screen',w_units='screen',w=40,h=40)
p.circle('x','y',source=flight_source,fill_color='black',hover_color='yellow',size=10,fill_alpha=0.8,line_width=0)

#HOVER INFORMATION AND LABEL
my_hover=HoverTool()
my_hover.tooltips=[('Call sign','@Callsign'),('Origin Country','@Origin_Country'),('Velocity(m/s)','@Velocity'),('Altitude(m)','@Baro_Altitude')]
labels = LabelSet(x='x', y='y', text='Callsign', level='glyph',
            x_offset=5, y_offset=5, source=flight_source, render_mode='canvas',background_fill_color='white',text_font_size="8pt")
p.add_tools(my_hover)
p.add_layout(labels)

output_notebook()

show(p)

Loading BokehJS ...